# Using a Deep Convolutional Neural Network to Recognize Fruits

In [6]:
import cv2
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from sklearn.metrics import confusion_matrix
import os
from keras.utils import to_categorical

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Loading Data
I'm using the [fruits360 dataset](https://www.kaggle.com/moltean/fruits) from kaggle. I only read in ten random fruits at a time in order to spare my wimpy computer.

In [4]:
# pick 10 random fruits based on their containing folders in the training data
n_fruits = 10 
prediction_fruits = np.random.choice(os.listdir('fruits-360/Training/'), n_fruits)
# alternately - load fruits from a prior random model run
#prediction_fruits = np.load('model_fruits.npy')

Display the fruits used for prediction -- there should have been 10 but I forgot to sample without replacement above

In [5]:
prediction_fruits

array(['Cherry Rainier', 'Granadilla', 'Guava', 'Mandarine',
       'Passion Fruit', 'Pear Abate', 'Pitahaya Red', 'Strawberry',
       'Tomato 1'], dtype='<U14')

In [8]:
# compile the file paths to all of the training/testing images for each type of fruit
train_img_urls = []
test_img_urls = []
train_labels = []
test_labels = []
train_prefix = 'fruits-360/Training/%s/'
test_prefix = 'fruits-360/Test/%s/'
for i in prediction_fruits:
    train_fruit_urls = [(train_prefix % i) + j for j in os.listdir(train_prefix % i)]
    test_fruit_urls = [(test_prefix % i) + j for j in os.listdir(test_prefix % i)]
    train_fruit_labels = [i] * len(train_fruit_urls)
    test_fruit_labels = [i] * len(test_fruit_urls)
    train_img_urls += train_fruit_urls 
    test_img_urls += test_fruit_urls 
    # collect the true labels (fruit name) for each training/testing sample
    train_labels += train_fruit_labels
    test_labels += test_fruit_labels

In [9]:
# read in the images as BGR (OpenCV does this instead of RGB for some reason) data 
train_images = np.stack([cv2.imread(i, 1) for i in train_img_urls])
test_images = np.stack([cv2.imread(i, 1) for i in test_img_urls])

Images are of size 100 x 100

In [7]:
print(train_images.shape)
print(test_images.shape)

(5398, 100, 100, 3)
(1818, 100, 100, 3)


### Data Setup

Map the fruit names to a one-hot encoding

In [11]:
label_mapping = {e:i for i, e in enumerate(np.unique(test_labels))}

In [13]:
label_mapping

{'Cherry Rainier': 0,
 'Granadilla': 1,
 'Guava': 2,
 'Mandarine': 3,
 'Passion Fruit': 4,
 'Pear Abate': 5,
 'Pitahaya Red': 6,
 'Strawberry': 7,
 'Tomato 1': 8}

In [16]:
train_int_labels = np.vectorize(label_mapping.get)(np.array(train_labels))
test_int_labels = np.vectorize(label_mapping.get)(np.array(test_labels))

In [11]:
train_one_hot_labels = to_categorical(train_int_labels, num_classes=n_fruits)
test_one_hot_labels = to_categorical(test_int_labels, num_classes=n_fruits)

### Model Setup
I used Keras to make a Convolutional Neural Network with the following structure:

100 x 100 x 3 image  
=>  
Convolutional Layer (32 filters, 3x3 kernel, stride 2)  
=>  
Convolutional Layer (32 filters, 3x3 kernel, stride 2)  
=>  
Pooling Layer (Max Pooling, 2x2)  
=>  
Dropout Layer (Keep 75%)  
=>  
Convolutional Layer (64 filters, 3x3 kernel, stride 1)  
=>  
Convolutional Layer (64 filters, 3x3 kernel, stride 1)  
=>  
Pooling Layer (Max Pooling, 2x2)  
=>  
Dropout Layer (Keep 75%)  
=>  
Convolutional Layer (16 filters, 3x3 kernel, stride 1)  
=>  
Convolutional Layer (16 filters, 3x3 kernel, stride 1)  
=>  
Pooling Layer (Max Pooling, 2x2)  
=>  
Dropout Layer (Keep 75%)  
=>  
Flatten  
=>  
Softmax output  

In [17]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu', input_shape=train_images.shape[1:]),
    Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), activation='relu'),

    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),   
    Dropout(0.25),
    
    Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'),
    Conv2D(filters=16, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(n_fruits, activation='softmax')    

])

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 50, 50, 32)        896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
__________

In [19]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(train_images, train_one_hot_labels, epochs=10, batch_size=75)
score = model.evaluate(test_images, test_one_hot_labels, batch_size=128)

Epoch 1/10
5398/5398 [==============================] - 319s 59ms/step - loss: 2.2373 - acc: 0.2544
Epoch 2/10
5398/5398 [==============================] - 312s 58ms/step - loss: 0.9713 - acc: 0.6078
Epoch 3/10
5398/5398 [==============================] - 312s 58ms/step - loss: 0.4984 - acc: 0.8194
Epoch 4/10
5398/5398 [==============================] - 318s 59ms/step - loss: 0.3351 - acc: 0.8824
Epoch 5/10
5398/5398 [==============================] - 311s 58ms/step - loss: 0.1579 - acc: 0.9457
Epoch 6/10
5398/5398 [==============================] - 308s 57ms/step - loss: 0.1174 - acc: 0.9598
Epoch 7/10
5398/5398 [==============================] - 313s 58ms/step - loss: 0.1013 - acc: 0.9654
Epoch 8/10
5398/5398 [==============================] - 270s 50ms/step - loss: 0.0892 - acc: 0.9696
Epoch 9/10
5398/5398 [==============================] - 250s 46ms/step - loss: 0.1052 - acc: 0.9676
Epoch 10/10
1818/1818 [==============================] - 28s 15ms/step


### Model evaluation

Score on the test set

In [21]:
score

[0.025848573993855158, 0.9895489548954896]

In [7]:
# alternately -- load a saved model
# model = load_model('deep-cnn-fruits.h5')

Convert the predictions to a one-dimensional vector and compute a confusion matrix

In [10]:
one_hot_predictions = np.round(model.predict(test_images))

In [14]:
pred_class = np.argmax(one_hot_predictions,axis=1)

In [17]:
confusion_matrix(y_pred=pred_class,y_true=test_int_labels)

array([[227,   0,   0,   0,  19,   0,   0,   0,   0],
       [  0, 166,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 166,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 166,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 166,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 166,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 166,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 164,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 246]])

Save the model and the fruits it was computed on

In [50]:
model.save('deep-cnn-fruits.h5')

In [51]:
np.save('model_fruits.npy', np.unique(test_labels))